In [6]:
from datetime import datetime, date

import pandas as pd

from log_info import update_log_df, call_update_log_df


pd.set_option('display.max_colwidth', 200)

In [7]:
log_df: pd.DataFrame = call_update_log_df()

Processing file /home/madlit/dev/nginx/access.log.4.gz
Processing file /home/madlit/dev/nginx/access.log.9.gz
Processing file /home/madlit/dev/nginx/access.log.2.gz
Processing file /home/madlit/dev/nginx/access.log.11.gz
Processing file /home/madlit/dev/nginx/access.log.6.gz
Processing file /home/madlit/dev/nginx/access.log.5.gz
Processing file /home/madlit/dev/nginx/access.log.13.gz
Processing file /home/madlit/dev/nginx/access.log.7.gz
Processing file /home/madlit/dev/nginx/access.log.3.gz
Processing file /home/madlit/dev/nginx/access.log.12.gz
Processing file /home/madlit/dev/nginx/access.log.1
Processing file /home/madlit/dev/nginx/access.log.10.gz
Processing file /home/madlit/dev/nginx/access.log.14.gz
Processing file /home/madlit/dev/nginx/access.log
Processing file /home/madlit/dev/nginx/access.log.8.gz


**Enter report dates**

**_______**

In [3]:
begin_date = date(year=2018, month=12, day=10)
end_date = date(year=2018, month=12, day=18)

**______**

In [5]:
log_df_filtered = log_df[(log_df['date'] >= begin_date) & (log_df['date'] <= end_date)]
report_columns = ['outer_request', 'resource_group', 'resource']
groupby_columns = ['outer_request', 'resource_group']
log_df_groped = log_df_filtered[report_columns].groupby(groupby_columns)
print(f'Download statistics from {str(begin_date)} to {str(end_date)}')
log_df_groped.count()

Download statistics from 2018-12-10 to 2018-12-18


resource
outer_request resource_group           
False         datasets             1118
              deeppavlov_data      2503
              ecommerce_skill       632
              embeddings            585
              faq                   795
              lang_models           149
              vectorizer             78
True          datasets              343
              deeppavlov_data       706
              deepreply_data          3
              ecommerce_skill         6
              embeddings            343
              faq                    54
              lang_models            12
              ner                     2
              vectorizer              7
              vemgmtss.html           1
              yad                    12
              yt_dl                   2